### Mol Bubi állomások
- Állomások neve, GPS koordináták
    - https://bkkfutar.docs.apiary.io/#reference/0/bicyclerental/bicyclerental
- Címek
    - Google Geocogind API
        - https://developers.google.com/maps/documentation/geocoding
    - Nominatim OpenStreetMap API 
        - https://nominatim.openstreetmap.org/

In [1]:
import requests
from time import sleep
import pandas as pd

In [2]:
def collect_address_osm(lati, longi):    
    url = f"https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lati}&lon={longi}"
    response = requests.get(url).json()
    try:
        postcode = response['address']['postcode']
    except:
        postcode = None
    try:
        city = response['address']['city']
    except:
        city = None
    try:
        district = response['address']['district']
        district = district.replace(". kerület", "")
        roman_dict = {'I':1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':'9', 'X':10,
                      'XI':'11', 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'XVII':17, 'XVIII':18, 'XIX':19, 
                      'XX':20, 'XXI':21, 'XXII':22, 'XXIII':23}
        district = roman_dict[district]
    except:
        district = None
    try:
        road = response['address']['road']
    except:
        road = None
    
    collected_address = {'postcode':postcode, 'city':city, 'district':district, 'road':road} 
    sleep(1)
    return collected_address

In [3]:
def collect_address_google(lati, longi, api_key):
    url=f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lati},{longi}&key={api_key}"
    response = requests.get(url).json()
    
    number = ""
    for i in range(3):
        if response['results'][i]['address_components'][0]['types'][0] != "street_number":
            continue
        else:
            try:
                number = response['results'][i]['address_components'][0]['long_name']
                street = response['results'][i]['address_components'][1]['long_name']
                district = response['results'][i]['address_components'][2]['long_name']
                city = response['results'][i]['address_components'][3]['long_name']            
                postcode = response['results'][i]['address_components'][5]['long_name']
                
                district = district.replace(". kerület", "")
                roman_dict = {'I':1, 'II':2, 'III':3, 'IV':4, 'V':5,
                              'VI':6, 'VII':7, 'VIII':8, 'IX':'9', 'X':10, 
                              'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 
                              'XVI':16, 'XVII':17, 'XVIII':18, 'XIX':19, 'XX':20,
                              'XXI':21, 'XXII':22, 'XXIII':23}
                district = roman_dict[district]
            except:
                osm = collect_address_osm(lati, longi)
                postcode = osm['postcode']
                city = osm['city']
                district = osm['district']
                street = osm['road']
            
            break
    
    if number == "":
        osm = collect_address_osm(lati, longi)
        postcode = osm['postcode']
        city = osm['city']
        district = osm['district']
        street = osm['road']
    
    address = {'postcode':postcode, 
               'city':city, 
               'district':district, 
               'street':street, 
               'number':number}    
    #print(address['postcode'], address['city'], address['district'], address['street'],address['number'])
    return address

In [4]:
with open(r'C:\vremek\bubi_api_key.txt', 'r') as f:
    key = f.read()
with open(r'C:\vremek\google_places_api_key.txt', 'r') as f:
    google_api_key = f.read()

version = "3"
appVersion = "apiary-1.0"
includeReferences = "true"
url = f"https://futar.bkk.hu/api/query/v1/ws/otp/api/where/bicycle-rental.json?key={key}&version={version}&appVersion={appVersion}&includeReferences={includeReferences}"
response = requests.get(url).json()

In [5]:
df = pd.DataFrame()

for i, station in enumerate(response['data']['list']):
    code = station['code']
    if code == "BIKE":
        continue
    else:        
        lat = station['lat']
        lon = station['lon']
        name = station['name']    
    
    #print(station['lat'], station['lon'], station['name'])
    address = collect_address_google(lat, lon, google_api_key)
    #print("---------------------------------------------------")
    
    new_row = pd.Series({'StationName': name,
                         'Latitude': lat,
                         'Longitude': lon,
                         'PostalCode': address['postcode'],
                         'City': address['city'],
                         'District': address['district'],
                         'Street': f"{address['street']} {address['number']}"
                         })
    
    df = pd.concat([df, new_row.to_frame().T], ignore_index=True)

In [6]:
df

,StationName,Latitude,Longitude,PostalCode,City,District,Street
0,Millenáris,47.51032,19.028615,1024,Budapest,2,Kis Rókus utca 7
1,Pázmány Péter sétány - Északi tömb,47.475277,19.061092,1117,Budapest,11,Pázmány Péter sétány 1/A
2,Kassák Lajos utca,47.52874,19.069095,1134,Budapest,13,Kassák Lajos utca 68
3,Rákosrendező vasútállomás,47.52551,19.088246,1142,Budapest,14,Szegedi út 76
4,Infopark - aluljáró,47.469366,19.059271,1117,Budapest,11,Neumann János utca
...,...,...,...,...,...,...,...
185,Vahot utca - Wartha Vince utca,47.465117,19.027727,1119,Budapest,11,Etele út 65
186,Pannónia utca - Raoul Wallenberg utca,47.514491,19.052535,1136,Budapest,13,Pannónia utca 15
187,Széchenyi fürdő,47.518845,19.081321,1146,Budapest,14,Állatkerti körút 12a
188,Üllői út - Ferenc Körút,47.4859,19.06948,1092,Budapest,8,Üllői út 41


In [7]:
df.to_csv(r'C:\vremek\adatok\tablak\molbubi_bkk_google_osm.csv', sep=';', index=True)